## ETL Processes

In [34]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [35]:
try:
    conn = psycopg2.connect("host=localhost dbname=sparkify_db user=postgres password= 707707")

except psycopg2.Error as e:
    print("Error: Could not make theconn.set_session(autocommit=True) connection to PostgreSQl Database")
    print(e)

In [36]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: could not get curser to the Database")
    print(e)

In [37]:
conn.set_session(autocommit=True)

In [38]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [39]:
song_files = get_files("./data/song_data")

In [40]:
filepath = song_files[1]

In [41]:
filepath

'C:\\Users\\Arqab Raza\\Documents\\Personal_Workspace\\Udacity\\Workspace\\Project\\Data Modeling With Postgres\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json'

In [42]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [43]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [44]:
cur.execute(songs_table_insert, song_data)

In [45]:
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [46]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [47]:
cur.execute(artists_table_insert, artist_data)
conn.commit()

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [48]:
log_files = get_files("./data/log_data")

In [49]:
log_files

['C:\\Users\\Arqab Raza\\Documents\\Personal_Workspace\\Udacity\\Workspace\\Project\\Data Modeling With Postgres\\data\\log_data\\2018\\11\\2018-11-01-events.json',
 'C:\\Users\\Arqab Raza\\Documents\\Personal_Workspace\\Udacity\\Workspace\\Project\\Data Modeling With Postgres\\data\\log_data\\2018\\11\\2018-11-02-events.json',
 'C:\\Users\\Arqab Raza\\Documents\\Personal_Workspace\\Udacity\\Workspace\\Project\\Data Modeling With Postgres\\data\\log_data\\2018\\11\\2018-11-03-events.json',
 'C:\\Users\\Arqab Raza\\Documents\\Personal_Workspace\\Udacity\\Workspace\\Project\\Data Modeling With Postgres\\data\\log_data\\2018\\11\\2018-11-04-events.json',
 'C:\\Users\\Arqab Raza\\Documents\\Personal_Workspace\\Udacity\\Workspace\\Project\\Data Modeling With Postgres\\data\\log_data\\2018\\11\\2018-11-05-events.json',
 'C:\\Users\\Arqab Raza\\Documents\\Personal_Workspace\\Udacity\\Workspace\\Project\\Data Modeling With Postgres\\data\\log_data\\2018\\11\\2018-11-06-events.json',
 'C:\\User

In [50]:
filepath = log_files[1]

In [51]:
filepath

'C:\\Users\\Arqab Raza\\Documents\\Personal_Workspace\\Udacity\\Workspace\\Project\\Data Modeling With Postgres\\data\\log_data\\2018\\11\\2018-11-02-events.json'

In [52]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
1,None,Logged In,Stefany,F,0,White,NaN,free,"Lubbock, TX",GET,Home,1540708070796,82,None,200,1541122176796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [53]:
df = df[df['page'] == 'NextSong']
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Dance Gavin Dance,Logged In,Marina,F,0,Sutton,218.46159,free,"Salinas, CA",PUT,NextSong,1541064343796,47,Uneasy Hearts Weigh The Most,200,1541127957796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,OutKast,Logged In,Ayleen,F,3,Wise,227.52608,free,"Columbia, SC",PUT,NextSong,1541085793796,70,Ova Da Wudz,200,1541184339796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like...",71
167,Bloodhound Gang,Logged In,Kaleb,M,0,Cook,260.20526,free,"Yuba City, CA",PUT,NextSong,1540679673796,53,Uhn Tiss Uhn Tiss Uhn Tiss,200,1541187675796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,54
168,Rammstein,Logged In,Isaac,M,0,Valdez,228.46649,free,"Saginaw, MI",PUT,NextSong,1541078099796,112,Adios,200,1541191023796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3
169,Gershon Kingsley,Logged In,Isaac,M,1,Valdez,146.57261,free,"Saginaw, MI",PUT,NextSong,1541078099796,112,Pop Corn,200,1541191251796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3


In [54]:
t = pd.to_datetime(df['ts'], unit='ms')
df['ts'] = t
t.head()

0   2018-11-02 01:25:34.796
2   2018-11-02 01:30:41.796
3   2018-11-02 01:34:17.796
4   2018-11-02 02:42:48.796
5   2018-11-02 03:05:57.796
Name: ts, dtype: datetime64[ns]

In [55]:
t

0     2018-11-02 01:25:34.796
2     2018-11-02 01:30:41.796
3     2018-11-02 01:34:17.796
4     2018-11-02 02:42:48.796
5     2018-11-02 03:05:57.796
                ...          
166   2018-11-02 18:45:39.796
167   2018-11-02 19:41:15.796
168   2018-11-02 20:37:03.796
169   2018-11-02 20:40:51.796
170   2018-11-02 20:43:17.796
Name: ts, Length: 155, dtype: datetime64[ns]

In [56]:
time_data = [t, t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month, t.dt.year, t.dt.weekday]
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [57]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-02 01:25:34.796,1,2,44,11,2018,4
2,2018-11-02 01:30:41.796,1,2,44,11,2018,4
3,2018-11-02 01:34:17.796,1,2,44,11,2018,4
4,2018-11-02 02:42:48.796,2,2,44,11,2018,4
5,2018-11-02 03:05:57.796,3,2,44,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [58]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [59]:
user_df = df[['userId','firstName','lastName','gender', 'level']]
user_df

,userId,firstName,lastName,gender,level
0,101,Jayden,Fox,M,free
2,83,Stefany,White,F,free
3,83,Stefany,White,F,free
4,66,Kevin,Arellano,M,free
5,48,Marina,Sutton,F,free
...,...,...,...,...,...
166,71,Ayleen,Wise,F,free
167,54,Kaleb,Cook,M,free
168,3,Isaac,Valdez,M,free
169,3,Isaac,Valdez,M,free


#### Insert Records into Users Table
Implement the `users_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [60]:
for i, row in user_df.iterrows():
    cur.execute(users_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [61]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (t[index], row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [62]:
conn.close()